In [1]:
import pandas as pd
import numpy as np
from EIMTC.preprocessing import OneHotEncoderEIMTC, M1CNNPreprocessing
from EIMTC.selection import train_test_split
from EIMTC.metrics import classification_report
from EIMTC.models import M1CNN

In [2]:
filepath = './data/flowpic_selected_flows_merged.csv'
df = pd.read_csv(filepath, usecols=['udps.n_bytes', 'encapsulation', 'traffic_type'])
df

,udps.n_bytes,encapsulation,traffic_type
0,"[48, 130, 0, 176, 2, 1, 0, 4, 6, 112, 117, 98,...",nonvpn,chat
1,"[23, 3, 3, 0, 36, 0, 0, 0, 0, 0, 0, 0, 10, 138...",nonvpn,chat
2,"[22, 3, 1, 0, 181, 1, 0, 0, 177, 3, 3, 252, 61...",nonvpn,chat
3,"[23, 3, 3, 0, 36, 0, 0, 0, 0, 0, 0, 0, 16, 80,...",nonvpn,chat
4,"[22, 3, 1, 0, 181, 1, 0, 0, 177, 3, 3, 199, 82...",nonvpn,chat
...,...,...,...
199,"[68, 111, 53, 57, 142, 157, 210, 145, 236, 72,...",nonvpn,audio
200,"[0, 1, 0, 100, 33, 18, 164, 66, 83, 54, 88, 11...",vpn,audio
201,"[169, 25, 2, 65, 31, 97, 146, 94, 166, 37, 204...",vpn,audio
202,"[169, 25, 2, 65, 31, 97, 146, 94, 166, 37, 204...",vpn,audio


In [3]:
def preprocessing(df):
    df.dropna(inplace=True)
    M1CNNPreprocessing.preprocess_features(dataframe=df)
    enc = OneHotEncoderEIMTC()
    df['encapsulation_ohc'] = list(enc.fit_transform(df['encapsulation']).toarray())
    return df

In [4]:
df = preprocessing(df)
df

,udps.n_bytes,encapsulation,traffic_type,encapsulation_ohc
0,"[48, 130, 0, 176, 2, 1, 0, 4, 6, 112, 117, 98,...",nonvpn,chat,"[1.0, 0.0]"
1,"[23, 3, 3, 0, 36, 0, 0, 0, 0, 0, 0, 0, 10, 138...",nonvpn,chat,"[1.0, 0.0]"
2,"[22, 3, 1, 0, 181, 1, 0, 0, 177, 3, 3, 252, 61...",nonvpn,chat,"[1.0, 0.0]"
3,"[23, 3, 3, 0, 36, 0, 0, 0, 0, 0, 0, 0, 16, 80,...",nonvpn,chat,"[1.0, 0.0]"
4,"[22, 3, 1, 0, 181, 1, 0, 0, 177, 3, 3, 199, 82...",nonvpn,chat,"[1.0, 0.0]"
...,...,...,...,...
199,"[68, 111, 53, 57, 142, 157, 210, 145, 236, 72,...",nonvpn,audio,"[1.0, 0.0]"
200,"[0, 1, 0, 100, 33, 18, 164, 66, 83, 54, 88, 11...",vpn,audio,"[0.0, 1.0]"
201,"[169, 25, 2, 65, 31, 97, 146, 94, 166, 37, 204...",vpn,audio,"[0.0, 1.0]"
202,"[169, 25, 2, 65, 31, 97, 146, 94, 166, 37, 204...",vpn,audio,"[0.0, 1.0]"


In [6]:
n_bytes = len(df['udps.n_bytes'].iloc[0])
n_classes = len(df['encapsulation'].unique())
model = M1CNN(payload_size=n_bytes, n_classes=n_classes)
print(n_bytes, n_classes)

784 2


In [7]:
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(
    optimizer='adam',
    loss=CategoricalCrossentropy()
)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(df['udps.n_bytes'].values, df['encapsulation_ohc'].values, 
    test_size=0.25,
    stratify=df['encapsulation'].values,
    random_state=42)

In [9]:
epochs = 10
batch_size = 128
model.fit(
    np.stack(x_train), 
    np.stack(y_train), 
    epochs=epochs, 
    batch_size=batch_size, 
    use_multiprocessing=True,
    workers=4,
    verbose=1
)

Epoch 1/10
2/2 [==============================] - 28s 73ms/step - loss: 67.0880
Epoch 2/10
2/2 [==============================] - 0s 9ms/step - loss: 217.9472
Epoch 3/10
2/2 [==============================] - 0s 7ms/step - loss: 9.6047
Epoch 4/10
2/2 [==============================] - 0s 0s/step - loss: 1.5330
Epoch 5/10
2/2 [==============================] - 0s 16ms/step - loss: 0.5647
Epoch 6/10
2/2 [==============================] - 0s 16ms/step - loss: 0.4938
Epoch 7/10
2/2 [==============================] - 0s 0s/step - loss: 0.4467
Epoch 8/10
2/2 [==============================] - 0s 16ms/step - loss: 0.4713
Epoch 9/10
2/2 [==============================] - 0s 16ms/step - loss: 0.4620
Epoch 10/10
2/2 [==============================] - 0s 10ms/step - loss: 0.4308


In [10]:
predictions = model.model.predict(np.stack(x_test))
predictions = np.argmax(predictions, axis=1)
y_test_true = np.argmax(np.stack(y_test), axis=1)

report = classification_report(y_test_true, predictions)
report

'              precision    recall  f1-score   support\n\n           0       0.71      0.69      0.70        29\n           1       0.61      0.64      0.62        22\n\n    accuracy                           0.67        51\n   macro avg       0.66      0.66      0.66        51\nweighted avg       0.67      0.67      0.67        51\n'

In [9]:
with open('m1cnn_iscx2016flowpic_encaps_report.txt', "w+") as f:
    f.write(report)
